In [40]:
import os
import pickle
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from typing import List, Tuple, Dict
ProgressBar().register()

## Split Target Df


In [38]:
target_df = pd.read_csv("data/02_primary/target.csv")
print(target_df.shape)
target_df = dd.from_pandas(target_df, npartitions=300)
target_df

(13668, 2)


,ISSUER_NAME,No.
npartitions=298,,
0,object,int64
46,...,...
...,...,...
13662,...,...
13667,...,...


In [39]:
target_df.to_csv('data/04_splits/*.csv', index=False)

[########################################] | 100% Completed |  1.6s
[########################################] | 100% Completed |  1.7s
[########################################] | 100% Completed |  1.8s
[########################################] | 100% Completed |  1.8s
[########################################] | 100% Completed |  1.8s
[########################################] | 100% Completed |  1.8s
[########################################] | 100% Completed |  1.9s


['/research/haohang/text_matching/data/04_splits/000.csv',
 '/research/haohang/text_matching/data/04_splits/001.csv',
 '/research/haohang/text_matching/data/04_splits/002.csv',
 '/research/haohang/text_matching/data/04_splits/003.csv',
 '/research/haohang/text_matching/data/04_splits/004.csv',
 '/research/haohang/text_matching/data/04_splits/005.csv',
 '/research/haohang/text_matching/data/04_splits/006.csv',
 '/research/haohang/text_matching/data/04_splits/007.csv',
 '/research/haohang/text_matching/data/04_splits/008.csv',
 '/research/haohang/text_matching/data/04_splits/009.csv',
 '/research/haohang/text_matching/data/04_splits/010.csv',
 '/research/haohang/text_matching/data/04_splits/011.csv',
 '/research/haohang/text_matching/data/04_splits/012.csv',
 '/research/haohang/text_matching/data/04_splits/013.csv',
 '/research/haohang/text_matching/data/04_splits/014.csv',
 '/research/haohang/text_matching/data/04_splits/015.csv',
 '/research/haohang/text_matching/data/04_splits/016.csv

## Preprocess Search df

In [41]:
search_df = pd.read_csv(os.path.join('data', '02_primary', 'search.csv'))
search_df.shape

(284035, 3)

In [23]:
to_remove_words_list = [
    "inc",
    "inc.",
    "incorporated",
    "llc",
    "l.l.c.",
    "l.l.c",
    "co.",
    "capital",
    "securities",
    "security",
    "s.a.",
    "s.a",
    "l.p.",
    "plc",
    "ag",
    "s.a.s",
    "s.a.s.",
    "s.p.a.",
    "s.p.a",
    "sa",
    "corporation",
    "ab",
    "trust",
    "capital",
    "limited",
    "corp",
    "ltd",
    "ltd.",
]

In [24]:
def preprocess(cur_txt: str, to_remove_words_list: List = to_remove_words_list) -> str:
    """
    Preprocess names before matching:
    1. remove commas
    2. remove words in to_remove_words_list

    Args:
        cur_txt (str): current name
        to_remove_words_list (List[srt]): list of words to remove

    Returns:
        str: preprocessed equity name
    """
    cur_txt = cur_txt.replace(",", "")
    cur_txt = cur_txt.split(" ")
    result_str_list = [
        cur_word.lower()
        for cur_word in cur_txt
        if cur_word.lower() not in to_remove_words_list
    ]

    return " ".join(result_str_list).strip()

In [26]:
search_df['preprocessed'] = search_df['Full Name'].apply(preprocess)
search_list = search_df['preprocessed'].tolist()

In [27]:
search_df_preprocessed2full_name = {cur_row['preprocessed']: cur_row['Full Name'] for _, cur_row in search_df.iterrows()}
search_df_full_name2id = {cur_row['Full Name']: cur_row['No. ID'] for _, cur_row in search_df.iterrows()}

In [29]:
# save as pickle
with open(os.path.join('data', '03_feature', 'search_list.pkl'), 'wb') as f:
    pickle.dump(search_list, f)
with open(os.path.join('data', '03_feature', 'search_df_preprocessed2full_name.pkl'), 'wb') as f:
    pickle.dump(search_df_preprocessed2full_name, f)
with open(os.path.join('data', '03_feature', 'search_df_full_name2id.pkl'), 'wb') as f:
    pickle.dump(search_df_full_name2id, f)